# Contents

This notebook contains 5 sections of code which is used to build and verify a graph database in Neo4j of the BART network.

* Data Preparation
* Queries for Building Graph Database
* Create Graph Database (Neo4j)
* Verify the Graph Database
* Enhance the Graph Database

# Included Modules and Packages

In [1]:
import csv
import neo4j

import math
import numpy as np
import pandas as pd

import psycopg2

from geographiclib.geodesic import Geodesic

In [2]:
# Function to run a select query and return rows in a pandas dataframe
# Pandas puts all numeric values from postgres to float
# If it will fit in an integer, change it to integer


def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [3]:
# Connect to Postgres

connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

# Create a cursor for the connection

cursor = connection.cursor()

In [4]:
# Connect to Neo4j

driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))
session = driver.session(database="neo4j")

In [5]:
# Neo4j helper functions

def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)
    
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")

def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.create('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost: ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1
            
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    print("\n-------------------------------------------------------------------------------")
    print("  Zip Codes within " + str(miles) + " mile(s) of " + station + " BART Station")
    print("-------------------------------------------------------------------------------\n")
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        print("     zip:", zip, "  population: ", f'{population:10,}')
        total_population += population
        
    
    print("\n-------------------------------------------------------------------------------")
    print("  Total Population: ", f'{total_population:10,}')
    print("-------------------------------------------------------------------------------")

In [6]:
# Function to read a csv file

def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

# Data Preparation
## Drop and Create Tables

In [7]:
connection.rollback()

query = """

drop table if exists stations;
drop table if exists lines;
drop table if exists travel_times;

"""

cursor.execute(query)

connection.commit()

In [8]:
connection.rollback()

query = """

create table stations (
  station varchar(32),
  latitude numeric(9,6),
  longitude numeric(9,6),
  transfer_time numeric(3),
  primary key (station)
);

"""

cursor.execute(query)

connection.commit()

In [9]:
connection.rollback()

query = """

create table lines (
  line varchar(6),
  sequence numeric(2),
  station varchar(32),
  primary key (line, sequence)
);

"""

cursor.execute(query)

connection.commit()

In [10]:
connection.rollback()

query = """

create table travel_times (
  station_1 varchar(32),
  station_2 varchar(32),
  travel_time numeric(3),
  primary key (station_1, station_2)
);

"""

cursor.execute(query)

connection.commit()

## Validate Files

In [11]:
my_read_csv_file('../data/stations.csv', 10)

['station', 'latitude', 'longitude', 'transfer_time']
['12th Street', '37.803608', '-122.272006', '282']
['16th Street Mission', '37.764847', '-122.420042', '287']
['19th Street', '37.807869', '-122.26898', '67']
['24th Street Mission', '37.752', '-122.4187', '277']
['Antioch', '37.996281', '-121.783404', '0']
['Ashby', '37.853068', '-122.269957', '299']
['Balboa Park', '37.721667', '-122.4475', '48']
['Bay Fair', '37.697', '-122.1265', '63']
['Berryessa', '37.368361', '-121.874655', '288']

Printed  10 lines of  51 total lines.


In [12]:
my_read_csv_file('../data/lines.csv', 10)

['line', 'sequence', 'station']
['blue', '1', 'Dublin']
['blue', '2', 'West Dublin']
['blue', '3', 'Castro Valley']
['blue', '4', 'Bay Fair']
['blue', '5', 'San Leandro']
['blue', '6', 'Coliseum']
['blue', '7', 'Fruitvale']
['blue', '8', 'Lake Merritt']
['blue', '9', 'West Oakland']

Printed  10 lines of  115 total lines.


In [13]:
my_read_csv_file('../data/travel_times.csv', 10)

['station_1', 'station_2', 'travel_time']
['12th Street', '19th Street', '120']
['12th Street', 'Lake Merritt', '180']
['12th Street', 'West Oakland', '300']
['16th Street Mission', '24th Street Mission', '120']
['16th Street Mission', 'Civic Center', '180']
['19th Street', 'MacArthur', '180']
['24th Street Mission', 'Glen Park', '180']
['Antioch', 'Pittsburg Center', '420']
['Ashby', 'Downtown Berkeley', '180']

Printed  10 lines of  52 total lines.


## Load Data to Tables

In [14]:
connection.rollback()

query = """

copy stations
from '/user/projects/ucb_mids_project_3/data/stations.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [15]:
connection.rollback()

query = """

copy lines
from '/user/projects/ucb_mids_project_3/data/lines.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [16]:
connection.rollback()

query = """

copy travel_times
from '/user/projects/ucb_mids_project_3/data/travel_times.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

## Verify Loads to Tables

In [17]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from stations
order by station

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282
1,16th Street Mission,37.764847,-122.420042,287
2,19th Street,37.807869,-122.268980,67
3,24th Street Mission,37.752000,-122.418700,277
4,Antioch,37.996281,-121.783404,0
5,Ashby,37.853068,-122.269957,299
6,Balboa Park,37.721667,-122.447500,48
7,Bay Fair,37.697000,-122.126500,63
8,Berryessa,37.368361,-121.874655,288
9,Castro Valley,37.690748,-122.075679,0


In [18]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from lines
order by lines, sequence

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,line,sequence,station
0,blue,1,Dublin
1,blue,2,West Dublin
2,blue,3,Castro Valley
3,blue,4,Bay Fair
4,blue,5,San Leandro
...,...,...,...
109,yellow,23,Daly City
110,yellow,24,Colma
111,yellow,25,South San Francisco
112,yellow,26,San Bruno


In [19]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from travel_times
order by station_1, station_2

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station_1,station_2,travel_time
0,12th Street,19th Street,120
1,12th Street,Lake Merritt,180
2,12th Street,West Oakland,300
3,16th Street Mission,24th Street Mission,120
4,16th Street Mission,Civic Center,180
5,19th Street,MacArthur,180
6,24th Street Mission,Glen Park,180
7,Antioch,Pittsburg Center,420
8,Ashby,Downtown Berkeley,180
9,Ashby,MacArthur,240


# Queries for Building Graph Database

In [20]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station
from stations
order by station

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station
0,12th Street
1,16th Street Mission
2,19th Street
3,24th Street Mission
4,Antioch
5,Ashby
6,Balboa Park
7,Bay Fair
8,Berryessa
9,Castro Valley


In [21]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station, line
from lines
where station = 'West Oakland'
order by 1, 2

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,line
0,West Oakland,blue
1,West Oakland,green
2,West Oakland,red
3,West Oakland,yellow


In [22]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station, line
from lines
order by 1, 2

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,line
0,12th Street,orange
1,12th Street,red
2,12th Street,yellow
3,16th Street Mission,blue
4,16th Street Mission,green
...,...,...
109,West Dublin,blue
110,West Oakland,blue
111,West Oakland,green
112,West Oakland,red


In [23]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines as a
join lines as b
    on a.station = b.station
join stations as s
    on a.station = s.station
where a.station = 'West Oakland'
    and a.line != b.line
order by 1, 2, 3

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,from_line,to_line,transfer_time
0,West Oakland,blue,green,283
1,West Oakland,blue,red,283
2,West Oakland,blue,yellow,283
3,West Oakland,green,blue,283
4,West Oakland,green,red,283
5,West Oakland,green,yellow,283
6,West Oakland,red,blue,283
7,West Oakland,red,green,283
8,West Oakland,red,yellow,283
9,West Oakland,yellow,blue,283


In [24]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines as a
join lines as b
    on a.station = b.station
join stations as s
    on a.station = s.station
where a.line != b.line
order by 1, 2, 3

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,from_line,to_line,transfer_time
0,12th Street,orange,red,282
1,12th Street,orange,yellow,282
2,12th Street,red,orange,282
3,12th Street,red,yellow,282
4,12th Street,yellow,orange,282
...,...,...,...,...
203,West Oakland,red,green,283
204,West Oakland,red,yellow,283
205,West Oakland,yellow,blue,283
206,West Oakland,yellow,green,283


In [25]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.line, 
    a.station as "from station", 
    b.station as "to station", 
    t.travel_time as "travel time in seconds"
from lines as a
join lines as b
    on a.line = b.line 
        and a.sequence = b.sequence - 1
join travel_times as t
    on (a.station = t.station_1 and b.station = station_2)
        or (a.station = t.station_2 and b.station = station_1)
Where a.station = 'West Oakland' 
    or b.station = 'West Oakland'
order by 1, 2, 3


"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,line,from station,to station,travel time in seconds
0,blue,Lake Merritt,West Oakland,360
1,blue,West Oakland,Embarcadero,420
2,green,Lake Merritt,West Oakland,360
3,green,West Oakland,Embarcadero,420
4,red,12th Street,West Oakland,300
5,red,West Oakland,Embarcadero,420
6,yellow,12th Street,West Oakland,300
7,yellow,West Oakland,Embarcadero,420


In [26]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.line, 
    a.station as "from station", 
    b.station as "to station", 
    t.travel_time as "travel time in seconds"
from lines as a
join lines as b
    on a.line = b.line 
        and a.sequence = b.sequence - 1
join travel_times as t
    on (a.station = t.station_1 and b.station = station_2)
        or (a.station = t.station_2 and b.station = station_1)
order by 1, 2, 3

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,line,from station,to station,travel time in seconds
0,blue,16th Street Mission,24th Street Mission,120
1,blue,24th Street Mission,Glen Park,180
2,blue,Balboa Park,Daly City,240
3,blue,Bay Fair,San Leandro,240
4,blue,Castro Valley,Bay Fair,240
...,...,...,...,...
103,yellow,Rockridge,MacArthur,240
104,yellow,San Bruno,SFO,240
105,yellow,South San Francisco,San Bruno,240
106,yellow,Walnut Creek,Lafayette,300


# Create Graph Database (Neo4j)

In [27]:
# Wipe the database and verify

my_neo4j_wipe_out_database()
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


## Queries to Build Graph Database

In [28]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 100
  Relationships: 0
-------------------------


In [29]:
connection.rollback()

query = """

select station, line
from lines
order by 1, 2

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    my_neo4j_create_node(line + " " + station)
    my_neo4j_create_relationship_one_way('depart ' + station, line + " " + station, 0)
    my_neo4j_create_relationship_one_way( line + " " + station, 'arrive ' + station, 0)
    
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 228
-------------------------


In [30]:
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines as a
join lines as b
    on a.station = b.station
join stations as s
    on a.station = s.station
where a.line != b.line
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    my_neo4j_create_relationship_one_way(from_line + " " + station, to_line + " " + station, transfer_time)
    
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 436
-------------------------


In [31]:
connection.rollback()

query = """

select a.line, 
    a.station as "from station", 
    b.station as "to station", 
    t.travel_time as "travel time in seconds"
from lines as a
join lines as b
    on a.line = b.line 
        and a.sequence = b.sequence - 1
join travel_times as t
    on (a.station = t.station_1 and b.station = station_2)
        or (a.station = t.station_2 and b.station = station_1)
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = row[1]
    to_station = row[2]
    travel_time = int(row[3])

    my_neo4j_create_relationship_two_way(line + " " + from_station, line + " " + to_station, travel_time)
    
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


# Verify the Graph Database

In [32]:
my_neo4j_shortest_path("depart Dublin", "arrive Antioch")


--------------------------------
   Total Cost:  5813
   Minutes:  96.9
--------------------------------
depart Dublin, 0, 0
blue Dublin, 0, 0
blue West Dublin, 180, 180
blue Castro Valley, 600, 780
blue Bay Fair, 240, 1020
blue San Leandro, 240, 1260
blue Coliseum, 240, 1500
orange Coliseum, 54, 1554
orange Fruitvale, 240, 1794
orange Lake Merritt, 300, 2094
orange 12th Street, 180, 2274
orange 19th Street, 120, 2394
orange MacArthur, 180, 2574
yellow MacArthur, 59, 2633
yellow Rockridge, 240, 2873
yellow Orinda, 300, 3173
yellow Lafayette, 300, 3473
yellow Walnut Creek, 300, 3773
yellow Pleasant Hill, 120, 3893
yellow Concord, 360, 4253
yellow North Concord, 180, 4433
yellow Pittsburg, 360, 4793
yellow Pittsburg Center, 600, 5393
yellow Antioch, 420, 5813
arrive Antioch, 0, 5813


In [33]:
my_neo4j_shortest_path("depart SFO", "arrive OAK")


--------------------------------
   Total Cost:  3882
   Minutes:  64.7
--------------------------------
depart SFO, 0, 0
yellow SFO, 0, 0
yellow San Bruno, 240, 240
yellow South San Francisco, 240, 480
yellow Colma, 180, 660
yellow Daly City, 240, 900
yellow Balboa Park, 240, 1140
green Balboa Park, 48, 1188
green Glen Park, 120, 1308
green 24th Street Mission, 180, 1488
green 16th Street Mission, 120, 1608
green Civic Center, 180, 1788
green Powell Street, 60, 1848
green Montgomery Street, 120, 1968
green Embarcadero, 60, 2028
green West Oakland, 420, 2448
green Lake Merritt, 360, 2808
green Fruitvale, 300, 3108
green Coliseum, 240, 3348
gray Coliseum, 54, 3402
gray OAK, 480, 3882
arrive OAK, 0, 3882


In [34]:
my_neo4j_shortest_path("depart Downtown Berkeley", "arrive Castro Valley")


--------------------------------
   Total Cost:  2214
   Minutes:  36.9
--------------------------------
depart Downtown Berkeley, 0, 0
orange Downtown Berkeley, 0, 0
orange Ashby, 180, 180
orange MacArthur, 240, 420
orange 19th Street, 180, 600
orange 12th Street, 120, 720
orange Lake Merritt, 180, 900
orange Fruitvale, 300, 1200
orange Coliseum, 240, 1440
blue Coliseum, 54, 1494
blue San Leandro, 240, 1734
blue Bay Fair, 240, 1974
blue Castro Valley, 240, 2214
arrive Castro Valley, 0, 2214


In [35]:
my_neo4j_shortest_path("depart San Bruno", "arrive San Leandro")


--------------------------------
   Total Cost:  3348
   Minutes:  55.8
--------------------------------
depart San Bruno, 0, 0
red San Bruno, 0, 0
red South San Francisco, 240, 240
red Colma, 180, 420
red Daly City, 240, 660
red Balboa Park, 240, 900
blue Balboa Park, 48, 948
blue Glen Park, 120, 1068
blue 24th Street Mission, 180, 1248
blue 16th Street Mission, 120, 1368
blue Civic Center, 180, 1548
blue Powell Street, 60, 1608
blue Montgomery Street, 120, 1728
blue Embarcadero, 60, 1788
blue West Oakland, 420, 2208
blue Lake Merritt, 360, 2568
blue Fruitvale, 300, 2868
blue Coliseum, 240, 3108
blue San Leandro, 240, 3348
arrive San Leandro, 0, 3348


In [36]:
my_neo4j_shortest_path("depart Embarcadero", "arrive Civic Center")


--------------------------------
   Total Cost:  240
   Minutes:  4.0
--------------------------------
depart Embarcadero, 0, 0
blue Embarcadero, 0, 0
blue Montgomery Street, 60, 60
blue Powell Street, 120, 180
blue Civic Center, 60, 240
arrive Civic Center, 0, 240


# Enhance the Graph Database

In [37]:
my_station_get_zips("Downtown Berkeley", 1)


-------------------------------------------------------------------------------
  Zip Codes within 1 mile(s) of Downtown Berkeley BART Station
-------------------------------------------------------------------------------

     zip: 94702   population:      17,092
     zip: 94703   population:      21,937
     zip: 94704   population:      29,190
     zip: 94709   population:      11,740
     zip: 94720   population:       2,971

-------------------------------------------------------------------------------
  Total Population:      82,930
-------------------------------------------------------------------------------


In [38]:
for i in range(1,6):
    my_station_get_zips("Downtown Berkeley", i)


-------------------------------------------------------------------------------
  Zip Codes within 1 mile(s) of Downtown Berkeley BART Station
-------------------------------------------------------------------------------

     zip: 94702   population:      17,092
     zip: 94703   population:      21,937
     zip: 94704   population:      29,190
     zip: 94709   population:      11,740
     zip: 94720   population:       2,971

-------------------------------------------------------------------------------
  Total Population:      82,930
-------------------------------------------------------------------------------

-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Downtown Berkeley BART Station
-------------------------------------------------------------------------------

     zip: 94618   population:      17,041
     zip: 94702   population:      17,092
     zip: 94703   population:      21,937
     zip: 94704   pop

In [39]:
my_station_get_zips("Antioch", 2)


-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Antioch BART Station
-------------------------------------------------------------------------------

     zip: 94509   population:      66,933

-------------------------------------------------------------------------------
  Total Population:      66,933
-------------------------------------------------------------------------------


In [40]:
my_station_get_zips("Powell Street", 2)


-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Powell Street BART Station
-------------------------------------------------------------------------------

     zip: 94102   population:      31,067
     zip: 94103   population:      28,735
     zip: 94104   population:         546
     zip: 94105   population:       9,155
     zip: 94107   population:      29,689
     zip: 94108   population:      14,550
     zip: 94109   population:      56,677
     zip: 94111   population:       3,620
     zip: 94114   population:      34,754
     zip: 94115   population:      35,004
     zip: 94123   population:      25,941
     zip: 94133   population:      26,527
     zip: 94158   population:       7,291

-------------------------------------------------------------------------------
  Total Population:     303,556
-------------------------------------------------------------------------------


In [41]:
my_station_get_zips("Coliseum", 2)


-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Coliseum BART Station
-------------------------------------------------------------------------------

     zip: 94601   population:      52,299
     zip: 94603   population:      34,593
     zip: 94613   population:         861
     zip: 94621   population:      35,287

-------------------------------------------------------------------------------
  Total Population:     123,040
-------------------------------------------------------------------------------


In [42]:
my_station_get_zips("Milpitas", 2)


-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Milpitas BART Station
-------------------------------------------------------------------------------

     zip: 95131   population:      31,170

-------------------------------------------------------------------------------
  Total Population:      31,170
-------------------------------------------------------------------------------


In [43]:
my_station_get_zips("Millbrae", 2)


-------------------------------------------------------------------------------
  Zip Codes within 2 mile(s) of Millbrae BART Station
-------------------------------------------------------------------------------

     zip: 94030   population:      22,710
     zip: 94128   population:         135

-------------------------------------------------------------------------------
  Total Population:      22,845
-------------------------------------------------------------------------------
